In [34]:
import requests
import pandas as pd
import time
from datetime import datetime
import plotly.express as px
from requests.utils import quote


In [ ]:

UA = "EditTrendBot/1.0 (contact@example.com)"
session = requests.Session()
session.headers.update({"User-Agent": UA, "Accept": "application/json"})


In [ ]:

def pageedits(site, page, start, end, editor_type="user"):
    start_ts = datetime.strptime(start, "%Y-%m-%d").strftime("%Y%m%d")
    end_ts   = datetime.strptime(end,   "%Y-%m-%d").strftime("%Y%m%d")
    encoded = quote(page, safe='')
    url = (
        f"https://wikimedia.org/api/rest_v1/metrics/edits/per-page/"
        f"{site}/{encoded}/{editor_type}/daily/{start_ts}/{end_ts}"
    )
    resp = session.get(url, timeout=30)
    resp.raise_for_status()
    data = resp.json()

    items = data.get("items", [])
    if not items or not items[0].get("results"):
        return pd.DataFrame(columns=["date", "edits", "page"])

    results = items[0]["results"]
    sample = results[0]
    key = "count" if "count" in sample else ("edits" if "edits" in sample else None)
    if key is None:
        raise KeyError(f"Pas de clé 'count' ou 'edits' dans {sample.keys()}")

    return pd.DataFrame({
        "date":  pd.to_datetime([r["timestamp"] for r in results], utc=True),
        "edits": [r.get(key, 0) for r in results],
        "page":  page
    })
    


In [ ]:
# Fetch edits for a list of pages
# and concatenate the results into a single DataFrame
def fetch_edit_pages(site, pages, start, end, editor_type="user"):
    dfs = []
    for p in pages:
        dfs.append(pageedits(site, p, start, end, editor_type))
        time.sleep(0.1)
    return pd.concat(dfs, ignore_index=True)



In [38]:

# Paramètres
site   = "fr.wikipedia.org"
pages  = ["Emmanuel Macron", "Marine Le Pen", "Jean-Luc Mélenchon"]
start  = "2025-1-01"
end    = "2025-05-01"
editor = "user"


In [39]:

# Exécution
df = fetch_edit_pages(site, pages, start, end, editor)
fig = px.line(df, x="date", y="edits", color="page",
              title=f"Évol. du nb. d’éditions — {site}",
              labels={"edits": "Éditions/jour", "date": "Date"})
fig.show()





